## Final Project Submission

Please fill out:
* Student name: 
* Student pace: self paced / part time / full time
* Scheduled project review date/time: 
* Instructor name: 
* Blog post URL:


# Project Overview

This project will employ exploratory data analysis to generate actionable insights that inform strategic decisions for business stakeholders.

# Business Understanding

As major companies increasingly invest in original video content, our company has decided to enter the entertainment industry by launching a new movie studio. However, the organization currently lacks experience in film production and does not have a clear strategy for deciding what types of movies to create.

The primary business goal of this project is to reduce the risk associated with entering the movie industry by using data-driven insights to understand which types of films are performing best at the box office today and identify patterns related to movie genre, rating, viewership, and finanical performance (income generation and production costs).

By analyzing current box office trends and successful films, this project will provide actionable insights that can guide the leadership of the new movie studio in making informed decisions about what kinds of movies to produce. These insights will help the studio prioritize film types that are more likely to achieve commercial success, maximize return on investment, and compete effectively with established studios.

Ultimately, the findings from this analysis will support strategic decision-making around film development, allowing the company to enter the market with a clearer understanding of audience demand and industry trends.

## Business Goal

To determine which movie genres are the most commercially successful and financially viable.

## Business Question

What movie genres are the most commercially successful and financially viable?

### Specific Questions:

1. Which genres are commonly produced?

2. Which genres received the highest audience ratings?

    * Were these differences statistically significant?

3. Which genres attracted the largest audiences/ viewership?

    * Were these differences statistically significant?

4. What was the production costs of the 5 most common movie genres?

5. What was the income generated from the 5 most common movie genres?

#### Sub-analysis

1. What is the relationship between production budget and ratings?

2. What is the relationship between production costs and gross income?


# Data Understanding

## Data Source and Description

For this project, we will leverage multiple data sources to identify the most promising types of films to produce:

1. IMDB – Provided in SQL database format, containing detailed movie metadata, ratings, and cast/crew information.

2. Box Office Mojo – CSV/TSV files with box office performance data, including domestic and international gross.

3. The Numbers – CSV/TSV files offering financial data such as production budgets, box office performance, and revenue trends.

These diverse sources will allow us to combine financial, audience, and critical metrics to generate actionable insights for the new movie studio.


# Data Preparation and Cleaning

For this section, we will analyze each Data Source individually and in each, we will aim to:

1. To read into the Data Source.

2. Select the Columns that we plan to use to answer our business questions.

3. Check through the data in the Sources to sort out any missing or duplicate data.

4. Save the 'New' Data of the 4 Data Sources as different pandas files.

In [1]:
# Import sqlite3, pandas and numpy to help in data cleaning

import sqlite3
import pandas as pd
import numpy as np


#### Starting off with the IMDB SQL file

In [2]:
# Starting off with the IMDB file which is an SQLite format
# We first need to unzip it using necessary libraries

import zipfile
from pathlib import Path

# Path to the zip file
zip_path = Path('C:/Users/PC/Desktop/movies_project/dsc-phase-2-project-v3/zippedData/im.db.zip')

# Folder to extract to
extract_folder = zip_path.parent  # same folder as the zip file

# Open and extract
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_folder)

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\PC\\Desktop\\movies_project\\dsc-phase-2-project-v3\\zippedData\\im.db.zip'

In [ ]:
# Next we read into the IMDB file by creating connection then using Pandas_SQL to read

# Create a connection

conn = sqlite3.connect('C:/Users/PC/Desktop/movies_project/dsc-phase-2-project-v3/zippedData/im.db')

cursor = conn.cursor()

# read into the file using Pandas-SQL query and see the tables that we have

tables = pd.read_sql_query("""
SELECT *
FROM sqlite_master
WHERE type = 'table';
""", conn)

print(tables)

    type           name       tbl_name  rootpage  \
0  table   movie_basics   movie_basics         2   
1  table      directors      directors         3   
2  table      known_for      known_for         4   
3  table     movie_akas     movie_akas         5   
4  table  movie_ratings  movie_ratings         6   
5  table        persons        persons         7   
6  table     principals     principals         8   
7  table        writers        writers         9   

                                                 sql  
0  CREATE TABLE "movie_basics" (\n"movie_id" TEXT...  
1  CREATE TABLE "directors" (\n"movie_id" TEXT,\n...  
2  CREATE TABLE "known_for" (\n"person_id" TEXT,\...  
3  CREATE TABLE "movie_akas" (\n"movie_id" TEXT,\...  
4  CREATE TABLE "movie_ratings" (\n"movie_id" TEX...  
5  CREATE TABLE "persons" (\n"person_id" TEXT,\n ...  
6  CREATE TABLE "principals" (\n"movie_id" TEXT,\...  
7  CREATE TABLE "writers" (\n"movie_id" TEXT,\n  ...  


In [ ]:
# From this IMDB SQL file, we are only interested in the averagerating and ordering columns
# We will extract these and store them as a pandas dataframe

IMDB_df = pd.read_sql_query("""
SELECT mb.movie_id, mb.genres, mr.movie_id, mr.averagerating, mr.numvotes, ma.ordering
FROM movie_basics as mb
JOIN movie_ratings as mr
ON mb.movie_id = mr.movie_id
JOIN movie_akas as ma
ON mr.movie_id = ma.movie_id
GROUP BY mb.genres;
""", conn)

# Let us check the IMDB_df to confirm that we have the columns of choice
IMDB_df.head() # great, we have created a pandas dataframe with only the columns we need from the SQL file

,movie_id,genres,movie_id,averagerating,numvotes,ordering
0,tt0253093,None,tt0253093,6.6,8,1
1,tt0364201,Action,tt0364201,6.4,16,1
2,tt6848590,"Action,Adult,Comedy",tt6848590,3.4,28,1
3,tt1136688,"Action,Adventure",tt1136688,3.3,1428,1
4,tt0448694,"Action,Adventure,Animation",tt0448694,6.6,133355,1


##### Data cleaning and saving of the IMDB_df

In [ ]:
# check for missing data
IMDB_df.isna().sum() # There is only 1 missing data in the genres.

# Checking the length of the IMDB_df
len(IMDB_df) # There is 915 data entries. So we will just drop the 1 missing value without any effect on analysis

# dropping the missing value and saving using inplace = True
IMDB_df.dropna(inplace = True)

# confirming that we have dropped the 1 missing value
IMDB_df.isna().sum() # We have dropped the missing value

movie_id         0
genres           0
movie_id         0
averagerating    0
numvotes         0
ordering         0
dtype: int64

In [ ]:
# Next we check for any duplicate data within the IMDB_df
IMDB_df.duplicated().sum() # There are no duplicated values

0

In [ ]:
# Checking the general information of the IMDB dataframe before saving it, and confirming that our values for average rating, num of votes and ordering are numerical
IMDB_df.info() # the IMDB_df data have the right data-type designation

<class 'pandas.core.frame.DataFrame'>
Int64Index: 914 entries, 1 to 914
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   movie_id       914 non-null    object 
 1   genres         914 non-null    object 
 2   movie_id       914 non-null    object 
 3   averagerating  914 non-null    float64
 4   numvotes       914 non-null    int64  
 5   ordering       914 non-null    int64  
dtypes: float64(1), int64(2), object(3)
memory usage: 50.0+ KB


In [ ]:
# To make analysis easier later and make pushing easier as well, we will save our IMDB_df as a csv file and place index = False to prevent duplication of indices
IMDB_df.to_csv('IMDB_data.csv', index = False)

#### Next we read into, clean and save our Box Office Mojo Data

In [ ]:
# read our csv file using pandas
box_office_df = pd.read_csv('C:/Users/PC/Desktop/movies_project/dsc-phase-2-project-v3/zippedData/bom.movie_gross.csv.gz')

# confirm that we have read into the file
box_office_df['title'].head()

# This data seems not to group the movies in genres but instead has individual movie titles. We will use our business understanding and research to classify our movie titles into movie genres later during analysis.

0                                    Toy Story 3
1                     Alice in Wonderland (2010)
2    Harry Potter and the Deathly Hallows Part 1
3                                      Inception
4                            Shrek Forever After
Name: title, dtype: object

In [ ]:
# Checking the columns in the box_office_df
box_office_df.columns # we are only interested in the title, domestic_gross and foreign_gross columns

# selecting the columns
box_office_df = box_office_df[['title', 'domestic_gross', 'foreign_gross']] # we have focused on the columns of interest


In [ ]:
# Checking the general information of the dataframe to confirm that the columns have the right data type designation

box_office_df.info() # All are okay apart from the foreign_gross which is designated as object yet it is expected to be numerical for us to analyze it
# we will first get rid of all missing values then convert the foreign_gross data to numerical

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   domestic_gross  3359 non-null   float64
 2   foreign_gross   2037 non-null   object 
dtypes: float64(1), object(2)
memory usage: 79.5+ KB


##### Data Cleaning and Saving the Box_office_df

In [ ]:
# checking for missing data and total length of the dataframe
box_office_df.isna().sum() # There are 28 domestic gross missing values and 1350 missing foreign gross values
len(box_office_df) # There are a total of 3387 data entries. The missing 1350 values make up approximately 40% of the total data. Replacing all this data with mean or median might distort the analysis, therefore we opt to drop all missing values.

# dropping missing values and saving using inplace = True.
box_office_df.dropna(inplace = True)

# confirming whether there are any remaining missing values
box_office_df.isna().sum() # All missing values are dropped.

title             0
domestic_gross    0
foreign_gross     0
dtype: int64

In [ ]:
# Converting foreign_gross from object to numerical data type
box_office_df['foreign_gross'] = box_office_df['foreign_gross'] .replace('[\$,]', '', regex=True).astype(float)

# confirming that the data types in the box_office df are now in order
box_office_df.info() #Achieved

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2009 entries, 0 to 3353
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           2009 non-null   object 
 1   domestic_gross  2009 non-null   float64
 2   foreign_gross   2009 non-null   float64
dtypes: float64(2), object(1)
memory usage: 62.8+ KB


In [ ]:
# saving the box_office_df for later analysis

box_office_df.to_csv('box_office_data', index = False)

#### Finally, we read into our final selected movie data source - The Numbers Data

In [ ]:
# Read using pandas
numbers_df = pd.read_csv("C:/Users/PC/Desktop/movies_project/dsc-phase-2-project-v3/zippedData/tn.movie_budgets.csv.gz", sep = ",", encoding = "latin1")

# check that our file has been read
numbers_df.head()

# Checking the columns in the data
numbers_df.columns

# select only the columns we need
numbers_df = numbers_df[['movie', 'production_budget', 'domestic_gross', 'worldwide_gross']]

##### Data cleaning and Saving the Numbers_df

In [ ]:
# checking for missing data
numbers_df.isna().sum() # There are no missing values

# Checking for duplicates
numbers_df.duplicated().sum() # There are no duplicated values

# checking into the datatypes of the columns before saving our data
numbers_df.info() # the production budget, domestic gross and worldwide gross are all objects. 

# converting objects into numerical data types
numbers_df['production_budget'] = numbers_df['production_budget'].replace('[\$,]', '', regex=True).astype(float)  
numbers_df['domestic_gross'] = numbers_df['domestic_gross'].replace('[\$,]', '', regex=True).astype(float)  
numbers_df['worldwide_gross'] = numbers_df['worldwide_gross'].replace('[\$,]', '', regex=True).astype(float)  

# recheck the datatypes to confirm the change
numbers_df.info() # Change has been effected

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   movie              5782 non-null   object
 1   production_budget  5782 non-null   object
 2   domestic_gross     5782 non-null   object
 3   worldwide_gross    5782 non-null   object
dtypes: object(4)
memory usage: 180.8+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   movie              5782 non-null   object 
 1   production_budget  5782 non-null   float64
 2   domestic_gross     5782 non-null   float64
 3   worldwide_gross    5782 non-null   float64
dtypes: float64(3), object(1)
memory usage: 180.8+ KB


In [ ]:
# Saving our edited Numbers_df

numbers_df.to_csv('Numbers_data')

# Data Analysis and Visualization

* Note: For the purposes of collaboration, the 3 csv files were created, therefore, in the subsequent sections, since the data is not located in a similar location for all collaborators, we will use panda's 'pd.read_csv' to read into our edited, cleaned, saved csv files before the start of analysis and visualization.

Business Question 1: Which genres are commonly produced?

In [ ]:
# Using the IMDB_df

In [ ]:
# Visualization

Business Question 2: Which genres received the highest audience rating? (average rating and number of votes)

In [ ]:
# Using the IMDB_df

In [ ]:
# Visualization

In [ ]:
# Using the ANOVA test to assess for statistical significance
# import necessary libraries

Business Question 3: Which genres had the largest viewership?

In [ ]:
# Using the IMDB_df

In [ ]:
# Visualization

In [ ]:
# Using ANOVA to assess for statistical significance
# confirm necessary library imported


Business Question 4: What was the production costs of the 5 most common movie genres?

In [ ]:
# Using data from numbers_df to determine which 5 movies had the highest vs lowest production costs
 

In [ ]:
# Visualization

Business Question 5: What was the income generated from the 5 most common movie genres?

In [ ]:
# Using data from the numbers_df

In [ ]:
# Visualization 

In [ ]:
# Using data from and box_office_df 

In [ ]:
# Visualization 

Sub-analysis (Regression analysis)

In [ ]:
# Assessing relationship between production budget and ratings?


# Checking whether our variables meet the assumptions for linear regression

# Linearity





# Independence of errors





# Normality





# Equality of Variance





# Regression plot





Intrepretation of the Regression results


######FILL HERE############

In [ ]:
# Assessing relationship between production budget and gross income generated?


# Checking whether our variables meet the assumptions for linear regression

# Linearity





# Independence of errors





# Normality





# Equality of Variance





# Regression plot





Intrepretation of the Regression results


######FILL HERE############

# Business Recommendation

Based on our results, we recommend:

1.

2.

3.

4.

5.


# Limitations

1. We left out the rotten tomatoes movie source since the rating column that we would have used from it had both numerical and non_numerical grade ratings that would have made data analysis difficult.

2. We also opted to leave out the tmdb movie source since the movie genres within it were coded under the ID column, and there was no references to use, to help us know which movie ID belonged to which movie genre.

# Further Suggestions